In [25]:


import ray
import pathlib
import sqlite3
import tqdm
import modin

import modin.pandas as pd

from rich import print as rprint



In [11]:
base_directory = pathlib.Path('/Volumes/photos-new')

old_lrcat_dir = base_directory / "OLD_LR" / "lightroom-catalog" 
old_lrcat = old_lrcat_dir / "catalog-2-v13.lrcat"

new_lrcat_dir = base_directory / "Lightroom" 
new_lrcat = new_lrcat_dir / "Lightroom Catalog-v13-3.lrcat"



In [12]:
def list_tables(lrcat: pathlib.Path):
    conn = sqlite3.connect(lrcat)
    cursor = conn.cursor()

    # Execute the query to get all table names
    cursor.execute("SELECT * FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    tnames = []
    for table in tables:
        tnames.append(table[1])
    conn.close()
    return tnames


In [13]:
def table_rows(lrcat, tables):
    conn = sqlite3.connect(lrcat)
    cursor = conn.cursor()
    
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table};")
        rows = cursor.fetchall()
        print(f'{table}: {rows}')
    conn.close()
    

In [14]:
if not ray.is_initialized():
    ray.init()

In [19]:
def create_con():
    return sqlite3.connect(old_lrcat)

In [22]:
help(pd.read_sql)

Help on function read_sql in module modin.pandas.io:

read_sql(sql, con, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None, dtype_backend: 'Union[DtypeBackend, NoDefault]' = <no_default>, dtype=None) -> 'DataFrame | Iterator[DataFrame]'
    Read SQL query or database table into a DataFrame.

    Notes
    -----
    See pandas API documentation for `pandas.read_sql <https://pandas.pydata.org/pandas-docs/version/2.2.3/reference/api/pandas.read_sql.html>`_ for more.



In [51]:
#images = pd.read_sql_table("Adobe_images", sqlite3.connect(old_lrcat))
connection = sqlite3.connect(old_lrcat)
images = pd.read_sql("SELECT * from Adobe_images", connection).drop(columns=['id_global'])
library_file = pd.read_sql("SELECT * from AgLibraryFile", connection).drop(columns=['id_global'])
library_folder = pd.read_sql("SELECT * from AgLibraryFolder", connection).drop(columns=['id_global'])
library_root_folder = pd.read_sql("SELECT * from AgLibraryRootFolder", connection).drop(columns=['id_global'])
harvested_exifdata = pd.read_sql("SELECT * from AgHarvestedExifMetaData", connection)
exif_camera_model =pd.read_sql("SELECT * from AgInternedExifCameraModel", connection)
exif_lens =pd.read_sql("SELECT * from AgInternedExifLens", connection)


Reason: To use the parallel implementation of `read_sql`, pass either a SQLAlchemy connectable, the SQL connection string, or a ModinDatabaseConnection with the arguments required to make a connection, instead of <class 'sqlite3.Connection'>. For documentation on the ModinDatabaseConnection, see https://modin.readthedocs.io/en/latest/supported_apis/io_supported.html#connecting-to-a-database-for-read-sql
Reason: To use the parallel implementation of `read_sql`, pass either a SQLAlchemy connectable, the SQL connection string, or a ModinDatabaseConnection with the arguments required to make a connection, instead of <class 'sqlite3.Connection'>. For documentation on the ModinDatabaseConnection, see https://modin.readthedocs.io/en/latest/supported_apis/io_supported.html#connecting-to-a-database-for-read-sql
Reason: To use the parallel implementation of `read_sql`, pass either a SQLAlchemy connectable, the SQL connection string, or a ModinDatabaseConnection with the arguments required to mak

In [46]:
help(images)

Help on DataFrame in module modin.pandas.dataframe object:

class DataFrame(modin.pandas.base.BasePandasDataset)
 |  DataFrame(data=None, index=None, columns=None, dtype=None, copy=None, query_compiler: 'BaseQueryCompiler' = None) -> 'None'
 |
 |  Modin distributed representation of ``pandas.DataFrame``.
 |
 |  Internally, the data can be divided into partitions along both columns and rows
 |  in order to parallelize computations and utilize the user's hardware as much as possible.
 |
 |  Inherit common for ``DataFrame``-s and ``Series`` functionality from the
 |  `BasePandasDataset` class.
 |
 |  Parameters
 |  ----------
 |  data : DataFrame, Series, pandas.DataFrame, ndarray, Iterable or dict, optional
 |      Dict can contain ``Series``, arrays, constants, dataclass or list-like objects.
 |      If data is a dict, column order follows insertion-order.
 |  index : Index or array-like, optional
 |      Index to use for resulting frame. Will default to ``RangeIndex`` if no
 |      ind

In [35]:
def create_old_conn():
    return sqlite3.connect(old_lrcat)

images = ray.data.read_sql("SELECT * from Adobe_images", create_old_conn)
library_file = ray.data.read_sql("SELECT * from AgLibraryFile", create_old_conn)
library_folder = ray.data.read_sql("SELECT * from AgLibraryFolder", create_old_conn)
library_root_folder = ray.data.read_sql("SELECT * from AgLibraryRootFolder", create_old_conn)
harvested_exifdata = ray.data.read_sql("SELECT * from AgHarvestedExifMetaData", create_old_conn)
exif_camera_model =ray.data.read_sql("SELECT * from AgInternedExifCameraModel", create_old_conn)
exif_lens =ray.data.read_sql("SELECT * from AgInternedExifLens", create_old_conn)



2025-05-05 10:06:59,007	INFO sql_datasource.py:145 -- Sharding is not supported. Falling back to reading all data in a single task.
2025-05-05 10:06:59,015	INFO sql_datasource.py:145 -- Sharding is not supported. Falling back to reading all data in a single task.
2025-05-05 10:06:59,023	INFO sql_datasource.py:145 -- Sharding is not supported. Falling back to reading all data in a single task.
2025-05-05 10:06:59,030	INFO sql_datasource.py:145 -- Sharding is not supported. Falling back to reading all data in a single task.
2025-05-05 10:06:59,037	INFO sql_datasource.py:145 -- Sharding is not supported. Falling back to reading all data in a single task.
2025-05-05 10:06:59,045	INFO sql_datasource.py:145 -- Sharding is not supported. Falling back to reading all data in a single task.
2025-05-05 10:06:59,052	INFO sql_datasource.py:145 -- Sharding is not supported. Falling back to reading all data in a single task.
